In [1]:
import asyncio, time, sys
import websockets, json
import numpy as np
from numpy_ringbuffer import RingBuffer

In [2]:
class Session:
    device_data = json.dumps({})
    uri = 'ws://localhost:8000'
    ws_timeout = 5.0
    
    def calculate_features(self):
        device_data = json.loads(self.device_data)
        print(device_data["A1"])

    async def ServerBIT_recieve(self, uri):
        async with websockets.connect(uri) as websocket:
            while True:
                self.device_data = await websocket.recv()
                self.calculate_features()
                await asyncio.sleep(0.0)
                
    def run(self):
        uri = self.uri
        print("Serving on {}".format(uri))
#         while len(json.loads(self.device_data)) == 0:
        try:
            asyncio.get_event_loop().run_until_complete(
                self.ServerBIT_recieve(uri))
#             asyncio.get_event_loop().run_forever()
        except Exception as e:
            print(e)
            pass

In [3]:
session = Session()
session.run()

Serving on ws://localhost:8000
[0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0]
[0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0]


KeyboardInterrupt: 

In [9]:
win_size=5

W_A = np.zeros(win_size) + 1
W_B = np.zeros(win_size)

r = RingBuffer(10, dtype=(np.float, win_size))